模型可以请求调用工具来执行任务，例如从数据库中获取数据、搜索网络或运行代码。工具是以下两项的配对：
- 一个模式（Schema），包含工具的名称、描述和/或参数定义（通常是 JSON 模式）。
- 一个用于执行的函数或协程。

为了让模型能够使用您定义的工具，必须使用 bind_tools 将其绑定。在后续的调用中，模型可以根据需要选择调用任何一个已绑定的工具。

后面的课程，我们会单独介绍工具调用的相关内容，这里只是简单介绍一下工具调用的基本流程，**需要明确的是。当你单独使用模型（而非通过框架）时，你需要自行执行该工具，并将结果返回给模型，供其在后续推理中使用。而当你使用封装过的框架时，框架会自动处理整个工具调用循环。**

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()  # 自动查找当前目录的 .env 文件

dashscope_api_key = os.getenv("DASHSCOPE_API_KEY")
dashscope_base_url = os.getenv("DASHSCOPE_BASE_URL")

In [3]:
from langchain_qwq import ChatQwen

model = ChatQwen(
    model="qwen-plus",
    api_key=dashscope_api_key,
    base_url=dashscope_base_url,
    temperature=0.7,
)

In [4]:
def get_weather(location: str) -> str:
    """获取指定地点的天气。"""
    return f"{location} 的天气是 20 摄氏度，晴朗。"

# 将工具绑定到模型上
model_with_tools = model.bind_tools([get_weather])

# 调用模型询问天气
response = model_with_tools.invoke("宁波的天气怎么样？")

# 遍历模型返回的工具调用信息
for tool_call in response.tool_calls:
    print(f"工具名称: {tool_call['name']}")
    print(f"参数: {tool_call['args']}")
    print(f"调用ID: {tool_call['id']}")

工具名称: get_weather
参数: {'location': '宁波'}
调用ID: call_c47bf5158508475b94f3ee


# 调用流程解析

1. 模型先生成自然语言回复（如 “我将为你获取宁波的天气”）。
2. 接着开始构造 Tool Call，即调用 get_weather 工具，分配tool_call id。
3. 如果是流式输出，工具调用的 JSON 参数是逐字生成的（如 {"location": "宁波"}），LangChain在解析过程中会逐步拼接这些片段，在完整 JSON 构造完成前，LangChain会将中间片段标记为invalid_tool_calls，因为此时参数还不完整、无法解析。
4. 如果仅仅使用model.invoke，模型会主动停止：response_metadata={'finish_reason': 'tool_calls', ...}，流程结束，等待工具的手动执行。